In [1]:
import json
import os
import sys

import boto3
import botocore

boto3_bedrock = boto3.client('bedrock-runtime')

In [2]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

In [3]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

In [4]:
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

In [5]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print(email)

Subject: Apology for the inconvenience

John Doe,

I am writing to express my sincere apologies for the inconvenience you experienced during your recent interaction with our customer support engineer. We understand that your experience was not up to our standards, and we take your feedback seriously.

I have thoroughly investigated the matter and identified the areas where we fell short. Our team is taking steps to improve our processes and training to ensure that we provide better service to our customers in the future.

I would like to offer you a free replacement or a refund for the product that was affected by the issue. Please let us know which option would be more convenient for you, and we will promptly take care of it.

Once again, please accept my sincere apologies for any inconvenience caused. We value your business and hope that you will give us another chance to provide you with excellent service.

Sincerely,
Bob, Customer Service Manager


In [6]:
output = []
try:
    
    response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

		**Chunk 1**

Subject: Apology for the inconvenience caused

Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the inconvenience caused by the service provided by our 

		**Chunk 2**
customer support engineer.

We value your feedback and take it seriously, and I want to assure you that we are committed to resolving this issue promptly. Our team has thoroughly investigated your concerns, and I want to assure you that we will take the necessary steps to prevent such incidents from happening in the future.

I understand that your experience with our

		**Chunk 3**
 service has left you feeling frustrated, and I apologize for any distress or inconvenience this may have caused you. We value your business and want to ensure that you have a positive experience with us.

Please know that we have already taken measures to address the problem. Our customer support engineer has been disciplined, and we will provide him with additi

		**Chunk 4**
o